In [3]:
import torch
import transformers
import pandas as pd
import numpy as np
import re
import gc
data = pd.read_csv('data_all.csv')

In [4]:
texts = data['text'].str.replace('[^А-я]', ' ', regex=True).str.lower()

In [5]:
seq = None
inputs = None
model = None
tokenizer = None
out = None
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_reserved()

0

In [6]:
list_of_arrays = []

In [7]:
model = transformers.AutoModel.from_pretrained('DeepPavlov/rubert-base-cased')
tokenizer = transformers.AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
device=torch.device("cuda:0")
model.to(device)
for i in range(data.shape[0]//100+1):
    with torch.no_grad():
        seq = list(texts.values[i*100:(i+1)*100])
        inputs = tokenizer(seq, return_tensors="pt", pad_to_max_length=True, max_length=128, truncation=True).to(device)
        out = model(**inputs)[0]
        out_np = out.detach().cpu().numpy()
        for arr in out_np:
            list_of_arrays.append(arr)
        gc.collect()
        torch.cuda.empty_cache()
        print(i)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/backup/asen/vitaly/ml/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2346: FutureWar

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83


In [8]:
arr = np.array([x.reshape((-1)) for x in list_of_arrays])

In [9]:
data_encoded = pd.DataFrame(arr, columns = list(range(arr.shape[1])))

In [10]:
import pickle
encodings = open('embeddings', 'wb')
pickle.dump(data_encoded, encodings)
encodings.close()